<a href="https://colab.research.google.com/github/kartik703/Resume-Matcher-and-Interview-Question-Generator/blob/main/CV_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run the app with ngrok
from pyngrok import ngrok
public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)
app.run()

ModuleNotFoundError: No module named 'pyngrok'

In [3]:
!pip install langchain faiss-cpu transformers sentence-transformers PyPDF2 pyngrok Flask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [4]:
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2spSTpuhUshKKVq7XVFQbeG2ejK_5pFx4CNCK9tnex9ZSER5y")

# Start ngrok tunnel
public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)

Public URL: https://5d74-34-73-61-222.ngrok-free.app


In [ ]:
# Install dependencies
!pip install langchain faiss-cpu transformers sentence-transformers PyPDF2 pyngrok Flask

# Import libraries
import os
from flask import Flask, request, jsonify, render_template
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import base64
from pyngrok import ngrok

# Create the templates folder and index.html file
os.makedirs("templates", exist_ok=True)
index_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Resume Matcher</title>
</head>
<body>
    <h1>Resume Matcher and Interview Question Generator</h1>
    <form id="resumeForm">
        <label for="job_description">Job Description:</label><br>
        <textarea id="job_description" name="job_description" rows="4" cols="50"></textarea><br><br>
        <label for="resume_file">Upload Resume (PDF):</label><br>
        <input type="file" id="resume_file" name="resume_file" accept=".pdf"><br><br>
        <button type="submit">Process</button>
    </form>
    <h2>Results</h2>
    <div id="results"></div>

    <script>
        document.getElementById("resumeForm").addEventListener("submit", async (e) => {
            e.preventDefault();
            const jobDescription = document.getElementById("job_description").value;
            const resumeFile = document.getElementById("resume_file").files[0];

            if (!resumeFile) {
                alert("Please upload a resume file");
                return;
            }

            const reader = new FileReader();
            reader.onload = async () => {
                const base64Resume = reader.result.split(",")[1];
                try {
                    const response = await fetch("/process", {
                        method: "POST",
                        headers: {
                            "Content-Type": "application/json",
                        },
                        body: JSON.stringify({
                            job_description: jobDescription,
                            resume_file: base64Resume,
                        }),
                    });
                    const data = await response.json();
                    if (data.error) throw new Error(data.error);

                    document.getElementById("results").innerHTML = `
                        <h3>Match Summary:</h3>
                        <p>${data.match_summary}</p>
                        <h3>Interview Questions:</h3>
                        <ul>${data.interview_questions.split('\n').map(q => q ? `<li>${q}</li>` : '').join('')}</ul>
                    `;
                } catch (error) {
                    document.getElementById("results").innerHTML = `<p style="color: red">Error: ${error.message}</p>`;
                }
            };
            reader.readAsDataURL(resumeFile);
        });
    </script>
</body>
</html>
"""
with open("templates/index.html", "w") as f:
    f.write(index_html_content)

# Initialize Flask app
app = Flask(__name__)

# Load Hugging Face LLM
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
llm_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Load embeddings model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

def parse_resume(file_path):
    """Extract text from a resume (PDF only)"""
    if not os.path.exists(file_path):
        raise ValueError(f"File path {file_path} does not exist.")
    if not file_path.lower().endswith(".pdf"):
        raise ValueError("Only PDF files are supported.")
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""]
    )
    return text_splitter.split_documents(pages)

def create_vector_store(resume_chunks):
    """Create FAISS vector store from extracted resume text"""
    return FAISS.from_documents(resume_chunks, embeddings)

def job_match(job_description, vector_store):
    """Find best matching resumes for a given job description"""
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})

    # Custom prompt template
    prompt_template = """Answer the question based on the resume context below.
    Summarize how the candidate's qualifications match the job description.
    Focus on technical skills, relevant experience, and education.

    Job Description: {job_desc}
    Context: {context}
    Answer:"""

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["job_desc", "context"]
    )

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )

    result = qa.invoke({"input_documents": [], "query": job_description})
    return result["result"], result["source_documents"]

def generate_interview_questions(full_resume_text):
    """Generate interview questions based on a candidate's resume"""
    # Tokenize and chunk the resume text
    tokens = tokenizer.encode(full_resume_text)
    max_tokens = 500
    chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]

    questions = []
    for chunk in chunks:
        chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
        prompt = f"""Generate 2 professional interview questions based on:
        {chunk_text}
        Focus on technical skills, projects, and work experience.
        Format each question on a new line."""

        response = llm_pipeline(
            prompt,
            max_length=512,
            num_return_sequences=1,
            truncation=True
        )
        if response and len(response) > 0:
            questions.extend(response[0]['generated_text'].split('\n'))

    # Clean and deduplicate questions
    clean_questions = [q.strip() for q in questions if q.strip()]
    unique_questions = list(set(clean_questions))
    return '\n'.join(unique_questions[:5])

@app.route("/", methods=["GET"])
def home():
    return render_template("index.html")

@app.route("/process", methods=["POST"])
def process():
    try:
        data = request.json
        if not data:
            return jsonify({"error": "No data provided"}), 400

        job_desc = data.get("job_description")
        resume_file = data.get("resume_file")

        if not job_desc or not resume_file:
            return jsonify({"error": "Missing job_description or resume_file"}), 400

        # Save and process the uploaded file
        temp_file = "temp_resume.pdf"
        try:
            with open(temp_file, "wb") as f:
                f.write(base64.b64decode(resume_file.encode("utf-8")))

            # Process resume
            resume_chunks = parse_resume(temp_file)
            if not resume_chunks:
                raise ValueError("No content found in resume")

            vector_store = create_vector_store(resume_chunks)
            match_summary, source_docs = job_match(job_desc, vector_store)

            # Use full resume text for questions
            full_resume_text = " ".join([doc.page_content for doc in resume_chunks])
            interview_questions = generate_interview_questions(full_resume_text)

            return jsonify({
                "match_summary": match_summary,
                "interview_questions": interview_questions
            })
        finally:
            if os.path.exists(temp_file):
                os.remove(temp_file)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run the app with ngrok
public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)
app.run(debug=False)

Device set to use cpu


Public URL: https://0296-34-73-61-222.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Mar/2025 00:24:50] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Mar/2025 00:26:01] "GET /?job_description=About+Zelp%0D%0AZELP+is+a+London-based+start-up+dedicated+to+mitigating+the+impact+of+the+livestock+industry+on+the+environment.+It+was+founded+in+2017+by+two+brothers+motivated+by+the+global+need+to+address+the+climate+crisis+and+the+urge+to+make+the+beef+and+dairy+industry+more+sustainable.+%0D%0A%0D%0AZELP+has+patented+a+wearable+device+for+cattle+that+neutralizes+livestock’s+emissions,+which+are+responsible+for+almost+10%25+of+the+total+greenhouse+gas+emissions+that+cause+global+warming.+The+deployment+of+our+device+on+a+global+scale+has+the+potential+to+reduce+greenhouse+gas+emissions+equivalent+to+eliminating+emissions+from+all+forms+

In [6]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
